# Time-Series Forecasting with Amazon SageMaker Autopilot

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/autopilot|autopilot_time_series.ipynb)

---

### Contents

1. [Introduction](#introduction)
1. [Setup](#setup)
1. [Model Training](#training)
1. [Real-Time Predictions (Inference)](#realtime)
1. [Batch Predictions (Inference)](#batch)


### 1. Introduction <a name='introduction'>

This notebook uses Amazon SageMaker Autopilot to train a time-series model and produce predictions against the trained model. At the top-level, customers provide a set of tabular historical data on S3 and make an API to train a model. Once the model has been trained, you can elect to produce prediction as a batch or via a real-time endpoint.</n></n>  As part of the training process, SageMaker Autopilot manages and runs multiple time series models concurrently. All of these models are combined into a single ensembled model which blends the candidate models in a ratio that minimizes forecast error. Customers are provided with metadata and models for the ensemble and all underlying candidate models too. SageMaker Autopilot orchestrates this entire process and provides several artifacts as a result.

These artifacts include: 
- backtest (holdout) forecasts per base model over multiple time windows,
- accuracy metrics per base model,
- backtest results and accuracy metrics for the ensembled model,
- a scaled explainability report displaying the importance of each covariate and static metadata feature.
- all model artifacts are provided as well on S3, which can be registered or use for batch/real-time inference

### 2. Setup <a name='setup'>

In [1]:
# Update boto3 using this method, or your preferred method
!pip install --upgrade boto3 --quiet
!pip install --upgrade sagemaker --quiet

In [3]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep
import datetime

region = boto3.Session().region_name
session = sagemaker.Session()

# Modify the following default_bucket to use a bucket of your choosing
bucket = session.default_bucket()
#bucket = 'my-bucket'
prefix = 'my-project-name'

role = get_execution_role()

# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
bucket

'sagemaker-us-east-1-174305766568'

We provide a sample set of data to accompany this notebook. You may use our synthetic dataset, or alter this notebook to accommodate your own data. As a note, the next cell will copy a file to your S3 bucket and prefix defined in the last cell. As an alternate, we provide a method to copy the file to your local disk too.

IMPORTANT: When training a model, your input data can contain a mixture of covariate and static item metadata. Take care to create future-dated rows that extend to the end of your prediction horizon. In the future-dated rows, carry your static item metadata and expected covariate values. Future-dated target-value (y) should be empty. Please download the example synthetic file using the S3 copy command in the next cell. You can observe the data programmatically or in a text editor as an example.

The structure of the CSV file provided is as follows:
- product_code (required: ItemIdentifierAttributeName)
- product_category (static, categorical feature describing product_code)
- product_subcategory (static, categorical feature describing product_code)
- location_code (GroupingAttributeNames column to get predictions at product_code + location_code)
- scaled_price (covariate)
- promotion_email (covariate)
- promotion_homepage (covariate)
- timestamp (required, TimestampAttributeName)
- unit_sales (required: TargetAttributeName)

In [4]:
s3 = boto3.resource('s3')
copy_source = {
    'Bucket': 'amazon-forecast-samples',
    'Key': 'autopilot/synthetic-food-demand.csv'
}

s3.meta.client.copy(copy_source, bucket, prefix+'/train/synthetic-food-demand.csv')

# Alternate, copy the file for local inspection/use.
#!aws s3 cp s3://amazon-forecast-samples/autopilot/synthetic-food-demand.csv .

### 3. Model Training <a name='training'>

Establish an AutoML training job name

In [24]:
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
auto_ml_job_name = "ts-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

AutoMLJobName: ts-20240530-172148


In [20]:
import pandas as pd
#time=pd.read_csv('s3://sagemaker-us-east-1-174305766568/my-project-name/train/Time_Series_Train.csv')
# time['snapshot_day']=pd.to_datetime(time['snapshot_day'])
time.to_csv('s3://sagemaker-us-east-1-174305766568/my-project-name/train/Time_Series_Train.csv',index=False)
time

,asin,product_type,type,brand,snapshot_day,gv
0,B0CPBW82R9,TEA,ThreeP,CHAYAM,2024-03-02,26.0
1,B0CPBW82R9,TEA,ThreeP,CHAYAM,2024-04-13,2.0
2,B0CPBW82R9,TEA,ThreeP,CHAYAM,2023-12-30,14.0
3,B0CPBW82R9,TEA,ThreeP,CHAYAM,2023-12-16,2.0
4,B0CPBW82R9,TEA,ThreeP,CHAYAM,2023-12-09,9.0
...,...,...,...,...,...,...
1070774,5023012034,HONEY,ThreeP,Fresh Produce,2022-06-18,875.0
1070775,5023012034,HONEY,ThreeP,Fresh Produce,2024-04-20,37.0
1070776,5023012034,HONEY,ThreeP,Fresh Produce,2024-03-02,75.0
1070777,5023012034,HONEY,ThreeP,Fresh Produce,2022-10-08,947.0


Define training job specifications. More information about [create_auto_ml_job_v2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job_v2.html) can be found in our SageMaker documentation.</n></n>This JSON body leverages the built-in sample data schema. Please consult the documentation to understand how to alter the parameters for your unique schema.

In [ ]:
'TimeSeriesForecastingJobConfig': {
            'FeatureSpecificationS3Uri': 'string',
            'CompletionCriteria': {
                'MaxCandidates': 123,
                'MaxRuntimePerTrainingJobInSeconds': 123,
                'MaxAutoMLJobRuntimeInSeconds': 123
            },
            'ForecastFrequency': 'string',
            'ForecastHorizon': 123,
            'ForecastQuantiles': [
                'string',
            ],
            'Transformations': {
                'Filling': {
                    'string': {
                        'string': 'string'
                    }
                },
                'Aggregation': {
                    'string': 'sum'|'avg'|'first'|'min'|'max'
                }
            },
            'TimeSeriesConfig': {
                'TargetAttributeName': 'string',
                'TimestampAttributeName': 'string',
                'ItemIdentifierAttributeName': 'string',
                'GroupingAttributeNames': [
                    'string',
                ]
            },
            'HolidayConfig': [
                {
                    'CountryCode': 'string'
                },
            ]
        },

In [22]:
input_data_config = [
    {  'ChannelType': 'training',
            'ContentType': 'text/csv;header=present',
            'CompressionType': 'None',
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://{}/{}/train/'.format(bucket, prefix),
            }
        }
    }
]

output_data_config = {'S3OutputPath': 's3://{}/{}/train_output'.format(bucket, prefix)}

optimizaton_metric_config = {'MetricName': 'AverageWeightedQuantileLoss'}

automl_problem_type_config ={
        'TimeSeriesForecastingJobConfig': {
           'CompletionCriteria': {
                'MaxCandidates': 1,
                'MaxRuntimePerTrainingJobInSeconds': 7200,
                'MaxAutoMLJobRuntimeInSeconds': 18000
            },
            'ForecastFrequency': 'W',
            'ForecastHorizon': 4,
            'ForecastQuantiles': ['p50','p60','p70','p80','p90'],
            'Transformations': {
            'Filling': {
                'gv': {
                    'middlefill' : 'zero',
                    'backfill' : 'zero'
                    },
                
                    }
            },
            'TimeSeriesConfig': {
                'TargetAttributeName': 'gv',
                'TimestampAttributeName': 'snapshot_day',
                'ItemIdentifierAttributeName': 'asin',
                
            },
    'HolidayConfig': [
                {
                    'CountryCode': 'IN'
                }]
        }
        }
    

With parameters now defined, invoke the [training job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job_v2.html) and monitor for its completion. You can expect the training to take about 1 hour.

In [25]:
sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLProblemTypeConfig = automl_problem_type_config,
    AutoMLJobObjective=optimizaton_metric_config,
    RoleArn=role
)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:174305766568:automl-job/ts-20240530-172148',
 'ResponseMetadata': {'RequestId': '1dae9344-99b8-4853-8580-3adc1adc8a54',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1dae9344-99b8-4853-8580-3adc1adc8a54',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '89',
   'date': 'Thu, 30 May 2024 17:21:55 GMT'},
  'RetryAttempts': 0}}

Next, we demonstrate a looping mechanism to query (monitor) job status. When the status is ```Completed```, you may review the accuracy of the model and decide whether to perform inference on a batch or real-time API basis as described in this notebook. Please consult documentation for [describe_auto_ml_job_v2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/describe_auto_ml_job_v2.html) as needed.

In [21]:
describe_response

{'AutoMLJobName': 'ts-20240530-170649',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:174305766568:automl-job/ts-20240530-170649',
 'AutoMLJobInputDataConfig': [{'ChannelType': 'training',
   'ContentType': 'text/csv;header=present',
   'CompressionType': 'None',
   'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-east-1-174305766568/my-project-name/train/'}}}],
 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-174305766568/my-project-name/train_output'},
 'RoleArn': 'arn:aws:iam::174305766568:role/service-role/AmazonSageMaker-ExecutionRole-20240509T161882',
 'AutoMLJobObjective': {'MetricName': 'AverageWeightedQuantileLoss'},
 'AutoMLProblemTypeConfig': {'TimeSeriesForecastingJobConfig': {'CompletionCriteria': {'MaxCandidates': 1,
    'MaxRuntimePerTrainingJobInSeconds': 7200,
    'MaxAutoMLJobRuntimeInSeconds': 18000},
   'ForecastFrequency': 'W',
   'ForecastHorizon': 48,
   'ForecastQuantiles': ['p50', 'p60', 'p70', 'p80', 'p9

In [ ]:
describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
       datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(180)

2024-05-30 17:22:14.846454 InProgress - PreTraining
2024-05-30 17:25:15.043453 InProgress - PreTraining
2024-05-30 17:28:15.275680 InProgress - PreTraining
2024-05-30 17:31:15.442477 InProgress - PreTraining
2024-05-30 17:34:15.671442 InProgress - TrainingModels
2024-05-30 17:37:15.873943 InProgress - TrainingModels
2024-05-30 17:40:16.040432 InProgress - TrainingModels
2024-05-30 17:43:16.275633 InProgress - TrainingModels
2024-05-30 17:46:16.424039 InProgress - TrainingModels
2024-05-30 17:49:16.631457 InProgress - TrainingModels
2024-05-30 17:52:16.818975 InProgress - TrainingModels
2024-05-30 17:55:17.040061 InProgress - TrainingModels
2024-05-30 17:58:17.251878 InProgress - TrainingModels
2024-05-30 18:01:17.476272 InProgress - TrainingModels


Once training is completed, you can use the describe function to iterate over model leaderboard results. Below is an example to use the best candidate in the subsequent inference phase. Please consult our documentation on [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) as needed.

In [27]:
best_candidate = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_containers = best_candidate['InferenceContainers'] 
best_candidate_name = best_candidate['CandidateName']

reponse = sm.create_model(
ModelName = best_candidate_name,
ExecutionRoleArn = role,
Containers = best_candidate_containers
)

print('BestCandidateName:',best_candidate_name)
print('BestCandidateContainers:',best_candidate_containers)

BestCandidateName: ts-20240530-172148-trial-me-1
BestCandidateContainers: [{'Image': '473393118567.dkr.ecr.us-east-1.amazonaws.com/autopilot-timeseries:latest', 'ModelDataUrl': 's3://sagemaker-us-east-1-174305766568/my-project-name/train_output/ts-20240530-172148/ts-20240530-172148-trial-TimeSeriesModelTraining/models/model-ensemble/full-dataset/ts-20240530-172148-me-1-aa880af29cb240a388ce9095e1f3dfbacf52f01/output/model.tar.gz', 'Environment': {}}]


In [28]:
sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)

{'AutoMLJobName': 'ts-20240530-172148',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:174305766568:automl-job/ts-20240530-172148',
 'AutoMLJobInputDataConfig': [{'ChannelType': 'training',
   'ContentType': 'text/csv;header=present',
   'CompressionType': 'None',
   'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
     'S3Uri': 's3://sagemaker-us-east-1-174305766568/my-project-name/train/'}}}],
 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-174305766568/my-project-name/train_output'},
 'RoleArn': 'arn:aws:iam::174305766568:role/service-role/AmazonSageMaker-ExecutionRole-20240509T161882',
 'AutoMLJobObjective': {'MetricName': 'AverageWeightedQuantileLoss'},
 'AutoMLProblemTypeConfig': {'TimeSeriesForecastingJobConfig': {'CompletionCriteria': {'MaxCandidates': 1,
    'MaxRuntimePerTrainingJobInSeconds': 7200,
    'MaxAutoMLJobRuntimeInSeconds': 18000},
   'ForecastFrequency': 'W',
   'ForecastHorizon': 4,
   'ForecastQuantiles': ['p50', 'p60', 'p70', 'p80', 'p90

### 4. Real-Time Predictions (Inference) <a name='realtime'>

If you want to perform real-time inference, review this section. If you want to perform batch processing, you may skip the real-time inference section and move to [Batch Predictions (Inference)](#batch).

Define a model, endpoint configuration and endpoint name using the candidate metadata. Consult [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint_config.html) documentation for more detail. Additionally, please adjust the ```InstanceType``` and ```InitialInstanceCount``` according to need.

IMPORTANT: The data you supply for inference must have at least four valid historical values for each time-series.

In [ ]:
endpoint_config_name = f"epc-{best_candidate_name}"
endpoint_name = f"ep-{best_candidate_name}"

production_variants = [
        {
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelName": best_candidate_name,
            "VariantName": "AllTraffic",
        }
    ]

epc_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants
)

Next, you can deploy a real-time endpoint using the [create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint.html
) API. See the documentation for more details and options.

In [ ]:
sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name)

Poll for endpoint to become ready to serve (InService)

In [ ]:
describe_response = sm.describe_endpoint(EndpointName=endpoint_name)

job_run_status = describe_response["EndpointStatus"]

while job_run_status not in ("Failed", "InService", "Stopped"):
    describe_response = sm.describe_endpoint(EndpointName=endpoint_name)
    job_run_status = describe_response["EndpointStatus"]

    print(
       datetime.datetime.now(), describe_response["EndpointStatus"])
    sleep(60)

The next cells help demonstrate opening a local CSV file for inference. Alternately, this data could come from S3, a database query or live application. In this example, the data is loaded into a Python memory object.

In [ ]:
# A small sample file that corresponds to the sample training dataset and trained model schema
!aws s3 cp s3://amazon-forecast-samples/autopilot/real-time-payload.csv ./real-time-payload.csv

In [ ]:
input_file = './real-time-payload.csv'
f=open(input_file,'r')
inference_data = f.read()
f.close()

Method to instantiate SageMaker runtime client and [invoke endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint.html). Please note the guidance given in the documentation page which says the response must be within 60 seconds of invocation. Response times are a function of payload size. You should take care to provide history for single time-series, and carefully testing the ability to perform predictions for more than one time-series at a time.

In [ ]:
sm_client = boto3.client('sagemaker-runtime')

In [ ]:
response = sm_client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= inference_data,
    ContentType = 'text/csv')

prediction = response['Body'].read().decode()

At this point, the results from the real-time API call are loaded into a variable called ```prediction```. You have several options on what you do with the results. A few are given here.

Optional: Example of saving the resulting real-time predictions to a local filesystem. Carefully plan your naming convention.

In [ ]:
output_file = 'real-time-prediction-output.csv'
f=open(output_file,'w')
f.write(prediction)
f.close()

Optional: Example of saving the resulting real-time predictions to a S3 object. Carefully plan your naming convention.

In [ ]:
output_file = 'real-time-prediction-output.csv'
key='s3://{}/{}/realtime_inference/{}'.format(bucket, prefix, output_file)

s3_client = boto3.client('s3')
s3_client.put_object(Body=prediction, Bucket=bucket, Key=key)

Optional: Example of loading predictions into a Panda dataframe

In [ ]:
import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(prediction), sep=',')
df.head(10)

Finally, you may also elect to place predictions in a SQS queue, post them to a stream, or post them to another real-time API according to need.

#### Cleanup Real-time Endpoint Resources

As needed, you can stop the endpoint and related billing costs as follows. When you need the endpoint again, you can follow the deployment steps again. Ideally, at a future time, another newer model is trained and able to be deployed as well.

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

Caution: Do not delete the model if you intend on testing batch transformation too. If you do delete the model, you may redeploy it as long as the model artifact exists on S3. The next cell provides the container and S3 location for your best candidate model.

In [ ]:
print('BestCandidateContainers:',best_candidate_containers)

In [ ]:
sm.delete_model(ModelName=best_candidate_name)

### 5. Batch Predictions (Inference) <a name='batch'>

Please review [service limits](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-service-restrictions-and-limits.html
) with batch transform. At the time of writing, the documentation says the maximum size of the input data per invocation is 100 MB. Translated, when working with 
datasets over 100MB, you will need to prepare your data by splitting/sharding into multiple files.
 Take care to ensure each file contains whole time series. One potential way to do this is to use
 a function that splits data on the item key, or similar.



Launch Batch Transformation Job using [create_transform_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_transform_job.html). The runtime of the job is a function of the size of your data and the InstanceType and InstanceCount provided. Once the task is complete, results are available on S3 at the declared ```S3OutputPath``` location. From there, you can use an event handler or other mechanism to consume the results.

In [ ]:
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
transform_job_name=f'{best_candidate_name}-' + timestamp_suffix
print("BatchTransformJob: " + transform_job_name)

The next cell downloads a dataset once again and this time places in a ```batch_transform/input``` folder. Ideally, this input dataset can be all of your time-series, or a fraction thereof. Please take care to ensure the dataset is within the limits described.

IMPORTANT: The data you supply for inference must have at least four valid historical values for each time-series.

In [ ]:
s3 = boto3.resource('s3')
copy_source = {
    'Bucket': 'amazon-forecast-samples',
    'Key': 'autopilot/synthetic-food-demand.csv'
}

s3.meta.client.copy(copy_source, bucket, prefix+'/batch_transform/input/synthetic-food-demand.csv')

In [ ]:
response = sm.create_transform_job(
    TransformJobName=transform_job_name, 
    ModelName=best_candidate_name,
    MaxPayloadInMB=0,
    ModelClientConfig={
        'InvocationsTimeoutInSeconds': 3600
    },
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://{}/{}/batch_transform/input/'.format(bucket, prefix)
            }
        },
        'ContentType': 'text/csv',
        'SplitType': 'None'
    },
    TransformOutput={
        'S3OutputPath': 's3://{}/{}/batch_transform/output/'.format(bucket, prefix),
        'AssembleWith': 'Line',
    },
    TransformResources={
        'InstanceType': 'ml.m5.12xlarge',
        'InstanceCount': 1
    }
    )

Poll for batch transformation job to complete. Once completed, resulting prediction files are available at the URI shown in the prior cell, ```S3OutputPath```. We use the API method [describe_transform_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/describe_transform_job.html) to complete this step.

In [ ]:
describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)

job_run_status = describe_response["TransformJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)
    job_run_status = describe_response["TransformJobStatus"]

    print(
       datetime.datetime.now(), describe_response["TransformJobStatus"]
    )
    sleep(60)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/autopilot|autopilot_time_series.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/autopilot|autopilot_time_series.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/autopilot|autopilot_time_series.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/autopilot|autopilot_time_series.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/autopilot|autopilot_time_series.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/autopilot|autopilot_time_series.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/autopilot|autopilot_time_series.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/autopilot|autopilot_time_series.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/autopilot|autopilot_time_series.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/autopilot|autopilot_time_series.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/autopilot|autopilot_time_series.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/autopilot|autopilot_time_series.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/autopilot|autopilot_time_series.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/autopilot|autopilot_time_series.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/autopilot|autopilot_time_series.ipynb)
